<a href="https://colab.research.google.com/github/davidho27941/ML_tutorial_notebook/blob/main/Python_decorator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 裝飾器及其應用


## 簡介

裝飾器（Decorator）是Python中的一個非常好用的功能。他是[剖面導向程式設計](https://zh.m.wikipedia.org/zh-tw/%E9%9D%A2%E5%90%91%E5%88%87%E9%9D%A2%E7%9A%84%E7%A8%8B%E5%BA%8F%E8%AE%BE%E8%AE%A1)（Aspect-oriented programming, AOP）的一個實現。在Python中，我們可以透過裝飾器來減少編寫重複的程式碼，也能透過裝飾器來在**不改變裝飾對象的功能**的前提之上，為裝飾對象新增功能以及特性。接下來我們將在本文中介紹裝飾器並實作其應用。

## 淺談裝飾器

如果把原本的一個函式當作一個**燈泡**，他的功能就是發出一道白光，那麽裝飾器就是如同**燈罩**一般的存在。一個白熾燈泡在套上了不同顏色的燈罩之後，其發出的光芒所帶有的顏色也會隨之改變，但白熾燈本身仍然只是持續的發出白色的光芒。裝飾器也是相同的存在。被裝飾器所裝飾的函式，其本身的功能並未改變，然而在執行上，會因為裝飾器的效果，而額外的表現出其他功能。

## 淺談Python中的物件與函式

在Python中，函式（Function）是作為[頭等物件](https://zh.wikipedia.org/wiki/%E9%A0%AD%E7%AD%89%E7%89%A9%E4%BB%B6)的存在。一個頭等物件可以作為參數被傳遞給其他的函式，或者是存入一個變數來進行使用。以下是一個範例：

* 將數個函式作為串列的元素進行儲存  
    假設我們今天有三個函式`A`, `B`, `C`，在被呼叫時分別會回應相對應的`a`, `b`, `c`字母。在Python中我們可以用下方的語法來建構這三個函式：

In [ ]:
def A():
    print('a')

def B():
    print('b')
    
def C():
    print('c')

我們可以透過將這三個函式存在一個串列`func_list`之中，並依序執行他們，來表現出函式可以被作為串列的元素存在這個特性。

In [ ]:
func_list = [A, B, C]
for func in func_list:
    func()

a
b
c


執行上方程式碼，你會發現`func()`會分別回傳`'a'`, `'b'`, `'c'`。這代表func將會作為串列元素中的`A`, `B`, `C`三個函式的替代，並發揮功能。

* 將函式作為其他函式的參數來使用  
    我們可以建立一個函式，其參數為一個函式，這個函式的功能就是將傳入的函式加以執行。在Python中，這也是可行的。假設我們今天建立的函式名為`wrapper`，其內容如下：

In [ ]:
def wrapper(func):
    func()

在上方的程式碼中，`wrapper`將會接收一個函數`func`並執行他。我們可以編寫一個會回應`Hello World`的函式並將其傳遞給`wrapper`來執行。

In [ ]:
def hello_world():
    print('Hello World!')

wrapper(hello_world)

Hello World!


執行上方的程式碼，會發現`wrapper`會執行`hello_world`函式的內容。

經由以上兩個簡單的範例，我們可以確認在Python中，函式確實可以作為一個對象被傳入其他函式，也能作為變數以及一個元素來進行使用。

## 再談裝飾器

其實在上一個章節的第二的範例就是裝飾器的實作了。  
![](https://i.imgur.com/sPBjHlX.png)  
（圖片來源：https://memes.tw/wtf/396975#! ）

其實裝飾器就是一個能夠包裝其他函式的存在，這也是為何我們將第二個範例稱作`wrapper`的原因。

你可能會想：「什麽，就這樣？」或是「這到底有什麽好特別的？」。但是在這看似簡單的運作，卻包含著一個很重要的概念――閉合性(Closure)。


## 閉合性

閉合性，簡單來說就是在Python的函式在建立時，會產生一個**Local Scope**，在**同一個Scope之中的變數，均可以被同一個Scope中的函式所存取**。

我們可以透過建立一個巢狀的函式來了解閉合性是怎樣的存在。

In [ ]:
def outer_func(mode='mode'):
    outer_var = 'outer'
    def inner_func():
        inner_var = 'inner'
        print(outer_var, inner_var, mode)
    return inner_func

我們可以引用在"Towards Data Science"的文章中的圖片來解析`outer_func`的結構。

![](https://i.imgur.com/edw2umE.png)  
（圖片來源：https://towardsdatascience.com/closures-and-decorators-in-python-2551abbc6eb6 ）

根據上圖，我們可以一一對應`outer_func`的各個元素：

變數方面：
* `outer_var`：`outer_func`的內建變數，同時是`inner_func`的`free variable`。
* `inner_var`：`inner_func`的內建變數。
* `mode`: `outer_func`的預設參數，同時是`inner_func`的`free variable`。

函式方面：
* `outer_func`：最外層的函式，內有一內建函式`inner_func`。
* `inner_func`：在`outer_func`之中的內建函式，也是相對於`outer_func`的`closure`。

在解析過後，我們可以預期以下幾件事：

1. `outer_func`具有一名為`inner_func`的閉包(Closure)。
2. 對於`inner_func`而言，有兩個自由變數(free variable)，分別為`mode`以及`outer_var`。
3. `inner_func`可以取用`outer_func`中的變數。

那們，我們可以建立一個基於`outer_func`的`func`物件，並查看`func`物件的`.__closure__`以及`.__code__.co_freevar`屬性來確認我們的預期是否正確。

> `.__closure__`以及`.__code__.co_freevars`分別可以讓我們存取到閉包中的物件以及閉包的自由變數。

In [ ]:
func = outer_func()
print(f"func object has a closure with content:\n  {func.__closure__}")

func object has a closure with content:
  (<cell at 0x7f1df2cc9e90: str object at 0x7f1e132ff6f0>, <cell at 0x7f1df2cc94d0: str object at 0x7f1e11659330>)


In [ ]:
print(f"func object has free variables:\n  {func.__code__.co_freevars}")


func object has free variables:
  ('mode', 'outer_var')


我們也可以透過迭代`func.__closure__`中的物件，並將這些物件與`func.__code__.co_freevars`中的物件一一進行對照。

In [ ]:
for idx in range(len(func.__code__.co_freevars)):
    print(func.__code__.co_freevars[idx], "=",
        func.__closure__[idx].cell_contents)

mode = mode
outer_var = outer


藉由閉合性的協助，我們可以設計裝飾器，並且在裝飾對象函式的同時，也能有各多元的變化。更重要的是，因為這些運作都是在裝飾器的**Local Scope**中執行，不用擔心裝飾器內部的變數以及函式會影響到外部的環境。

由於閉包以及閉合性並非本文的重點。如果想進一步了解，可以參考以下來源：
1. https://steam.oxxostudio.tw/category/python/basic/closure.html
2. https://towardsdatascience.com/closures-and-decorators-in-python-2551abbc6eb6
3. https://super9.space/archives/1451
4. https://medium.com/citycoddee/python%E9%80%B2%E9%9A%8E%E6%8A%80%E5%B7%A7-4-lambda-function-%E8%88%87-closure-%E4%B9%8B%E8%AC%8E-7a385a35e1d8

## 語法糖與裝飾器

在Python中，有許多好用的語法糖，諸如列表推導式以及`with`相關的語法糖。而裝飾器也有一個。我們可以透過在被裝飾的函式前面加一個`@<裝飾器名稱>`來省去在先前範例中的傳遞語法。以下兩個範例是有無使用語法糖的實作方法：

* 未使用語法糖

In [ ]:
def wrapper(func):
    print("Decorator: I'm here.")
    func()

def hello_world():
    print('Hello World!')
    
wrapper(hello_world)

* 使用語法糖


In [ ]:
def wrapper(func):
    print("Decorator: I'm here.")
    func()

@wrapper
def hello_world():
    print('Hello World!')

以上兩種方法都能在執行後得到相同的結果。這就是裝飾器最基本的用法。藉由裝飾器的協助，我們可以簡化程式碼的編排，也能降低出現重複程式碼的頻率。

這就是裝飾器最基本的用法。

## 讓裝飾器可以傳遞被包裝對象的參數

在先前的範例中，我們只用裝飾器包裝不需要參數的函式。但並未示範如何包裝需要參數的函式，以及如何將被包裝對象的參數經由裝飾器傳遞給被包裝的對象。接下來我們將示範如何經由裝飾器傳遞被包裝對象的參數。

方法很簡單，**只需要多加一層裝飾器，並利用(\*args, \*\*kargs)進行參數的傳遞即可**。  
詳細程式碼如下方所示：


In [ ]:
def wrapper(func):
    def argument_passer(*args, **kargs):
        func(*args, **kargs)
    return argument_passer

@wrapper
def hello_sth(sth):
    print(f'Hello {sth}')

hello_sth(123)

Hello 123


如範例所示，只要在裝飾器中多設定一個函式，並接收(\*args, \*\*kargs)作為參數，即可將被裝飾對象所需要的函式傳遞進去。

## 讓裝飾器可以接受參數並改變行為
在先前的範例中，我們示範了如何使用裝飾器來改變被裝飾對象的行為。但如果只能單調的依照某一種規則來改變行為，那是不是為了好幾種不同的裝飾方法，就需要準備各種不同的裝飾器呢？幸好，在Python中不需要這麽做。我們可以透過再多一層的包裝，來令裝飾器能夠接受參數，並依照輸入的參數來決定裝飾器的行為。我們可以設計一個裝飾器，讓裝飾器在接受到參數所提示的模式之後，依照指定的模式來輸出我們指定的內容。

In [12]:
def wrpper_with_argument(mode='dry-run'):
    def wrapper(func):
        def argument_passer(*args, **kargs):
            if mode == 'dry-run':
                print('Running in dry-run mode.')
            else:
                print(f"Running function: {func.__name__}")
                func(*args, **kargs)
        return argument_passer
    return wrapper

以上是一個裝飾器，這個裝飾器能夠根據接受的參數`mode`來決定是否執行被包裝對象的函式。我們可以用先前的`hello_sth`函式來測試這個裝飾器是否可以依照我們設計的規則來執行。

In [13]:
@wrpper_with_argument(mode='dry-run')
def hello_sth_dry_run(sth):
    print(f'Hello {sth}!')

hello_sth_dry_run(123)

Running in dry-run mode.


很好，我們成功利用額外一層的包裝，讓裝飾器可以接收參數並改變自身行為。在上方的實作中，有一個缺陷：函式被裝飾後，函式本身的`__name__`屬性會被改寫。我們可以透過呼叫`hello_sth_dry_run`的`__name__`屬性來觀察此現象。

In [14]:
hello_sth_dry_run.__name__

'argument_passer'

為了解決這問題，我們可以透過使用`functools`函式庫的`wraps`函式來裝飾我們的`argument_passer`來解決這問題。`functools.wraps`函式透過`functools.partial`（文件[連結](https://docs.python.org/3/library/functools.html#functools.partial)）以及`functools.update_wrapper`（文件[連結](https://docs.python.org/3/library/functools.html#functools.update_wrapper)）來實作。

我們可以簡單的介紹一下`partial`以及`update_wrapper`函式：

* `partial`函式可以將函式的部份參數固定，並允許使用者只更動剩餘的參數來減少需要輸入的參數。
* `update_wrapper`的作用是將被包裝對象的屬性複製到`wrapper`函式之中。

`functools.wraps`的核心實作便是藉由`partial`來對`update_wrapper`進行包裝，並固定`wrapped`、`assigned`以及`updated`三個參數。藉由這樣的操作，透過對原函式的屬性進行複製，來達成減少裝飾器對於原函式的影響。  

那麽，我們就來將`functools.wraps`裝飾在`argument_passer`並觀察裝飾後的改變吧！



In [17]:
from functools import wraps

def wrpper_with_argument_new(mode='dry-run'):
    def wrapper(func):
        @wraps(func)
        def argument_passer(*args, **kargs):
            if mode == 'dry-run':
                print('Running in dry-run mode.')
            else:
                print(f"Running function: {func.__name__}")
                func(*args, **kargs)
        return argument_passer
    return wrapper

@wrpper_with_argument_new(mode='dry-run')
def hello_sth_dry_run(sth):
    print(f'Hello {sth}!')

print(f"The attribute '__name__' of 'hello_sth_dry_run': {hello_sth_dry_run.__name__}")

The attribute '__name__' of 'hello_sth_dry_run': hello_sth_dry_run


可以發現，我們現在可以正確的讓被包裝的對象擁有正確的屬性了。

## 結語

裝飾器對於Python使用者來說，是可以大幅減少程式碼並在最小的成本上發揮最大效果的一個特性。如果能夠善加利用，可以讓自己的專案事半功倍。希望這篇文章能夠讓讀者更加了解裝飾器的特性。